## imports ##

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
#remove warnings
import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('/Users/gilbenor/Documents/code_projects/msc/counting_research_algorithms/fifty_one/measurements/results/analysis/error_flags_analysis_body_all_mean.csv')
df.head()

,Label,PrawnID,Pond_Type,mean_length,std_length,Length_fov(mm),mean_pixels,pred_Distance_pixels,Length_ground_truth_annotation_pixels,mean_scale,pred_scale
0,full body:undistorted_GX010077_14_237.jpg_gamma,Prawn_0,circle_female,75.624000,3.484291,88.223862,397.144449,475.986860,381.935664,52.616333,53.952168
1,full body:undistorted_GX010077_14_237.jpg_gamma,Prawn_1,circle_female,119.409000,7.534347,130.307259,626.987760,701.859664,703.474985,52.616333,53.861901
2,full body:undistorted_GX010077_14_237.jpg_gamma,Prawn_2,circle_female,104.471333,11.118477,121.385411,551.072805,656.609979,580.506159,52.616333,54.092990
3,full body:undistorted_GX010067_33_625.jpg_gamma,Prawn_0,circle_female,120.649333,11.749097,139.780255,700.400878,748.301965,684.970469,58.258000,53.534168
4,full body:undistorted_GX010067_33_625.jpg_gamma,Prawn_1,circle_female,93.004000,6.587385,102.721787,540.592735,551.039760,531.465367,58.258000,53.643903


## Error decompoition ##

### cancellation magnitude ###

In [17]:
%%capture cap

figs = []

from pathlib import Path

out_root = Path("/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/phd/"
                "paper/images/final images/body")
out_root.mkdir(parents=True, exist_ok=True)

# Iterate over each unique pond type in the DataFrame
for pond_type in df['Pond_Type'].unique():
    # Filter the DataFrame for the current pond type and reset the index
    df_pond = df[df['Pond_Type'] == pond_type].reset_index(drop=True)
    
    # Calculate pixel error in millimeters
    df_pond['pixel_error_mm'] = (df_pond['mean_pixels'] - df_pond['pred_Distance_pixels']) * (1/df_pond['mean_scale']) * 10
    
    # Calculate scale error in millimeters
    df_pond['scale_error_mm'] = df_pond['mean_pixels'] * (1/df_pond['mean_scale'] - 1/df_pond['pred_scale']) * 10
    
    # Identify points where pixel and scale errors partially cancel each other
    partial_cancel_mask_all = (df_pond['pixel_error_mm'] * df_pond['scale_error_mm'] < 0)
    
    # Calculate the percentage of points with partial cancellation
    percent_partial_cancel_all = 100 * partial_cancel_mask_all.sum() / len(df_pond) if len(df_pond) > 0 else 0
    
    # Print analysis results for the current pond type
    print("\n--- Error Analysis for Pond Type: {} ---".format(pond_type))
    print("   - Number of points in df_pond: {}".format(len(df_pond)))
    print("1. Partial Cancellation of Errors:")
    print("   - Percentage of points with partial cancellation: {:.2f}%".format(percent_partial_cancel_all))
    num_partial_cancel_all = partial_cancel_mask_all.sum()
    print("   - Number of points with partial cancellation: {}".format(num_partial_cancel_all))
    
    # Calculate uncancelled and actual total errors for points with partial cancellation
    uncancelled_error = abs(df_pond[partial_cancel_mask_all]['pixel_error_mm']) + abs(df_pond[partial_cancel_mask_all]['scale_error_mm'])
    actual_total_error = abs(df_pond[partial_cancel_mask_all]['pixel_error_mm'] + df_pond[partial_cancel_mask_all]['scale_error_mm'])
    
    # Calculate the magnitude of error cancellation
    cancellation_magnitude = uncancelled_error - actual_total_error
    
    # Print cancellation magnitude statistics
    print("\n2. Cancellation Magnitude:")
    print("   - Mean cancellation magnitude: {:.2f}".format(cancellation_magnitude.mean()))
    print("   - Standard deviation of cancellation magnitude: {:.2f}".format(cancellation_magnitude.std()))
    print("   - Mean percentage of cancellation magnitude relative to Length_fov(mm): {:.2f}%".format(
        cancellation_magnitude.mean() / df_pond["Length_fov(mm)"].abs().mean() * 100))
    print("   - Standard deviation of relative cancellation magnitude: {:.2f}%".format(cancellation_magnitude.std() / df_pond["Length_fov(mm)"].abs().mean() * 100))
  
    print("\n----------------------------------------\n")

    # Calculate actual error and cancellation in millimeters for points with partial cancellation
    df_pond['actual_error_mm'] = abs(df_pond[partial_cancel_mask_all]['pixel_error_mm'] + df_pond[partial_cancel_mask_all]['scale_error_mm'])
    df_pond['cancellation_mm'] = abs(df_pond[partial_cancel_mask_all]['pixel_error_mm']) + abs(df_pond[partial_cancel_mask_all]['scale_error_mm']) - df_pond[partial_cancel_mask_all]['actual_error_mm']

    # Calculate and print the correlation between actual error and cancellation
    print(f"Correlation between actual error and cancellation: {df_pond['actual_error_mm'].corr(df_pond['cancellation_mm'])}")
     # Calculate the dominance of scale and pixel errors
    df_pond['dominant_error'] = np.where(
        abs(df_pond['scale_error_mm']) > abs(df_pond['pixel_error_mm']),
        'Scale Error',
        'Pixel Error'
    )

    # Count the number of occurrences for each dominant error type
    dominant_counts = df_pond['dominant_error'].value_counts()

    # Calculate the percentage of each dominant error type
    dominant_percentages = df_pond['dominant_error'].value_counts(normalize=True) * 100

    # Print the results
    print("\n3. Dominance of Error Types:")
    print("   - Number of points where Scale Error is dominant: {}".format(dominant_counts.get('Scale Error', 0)))
    print("   - Percentage of points where Scale Error is dominant: {:.2f}%".format(dominant_percentages.get('Scale Error', 0)))
    print("   - Number of points where Pixel Error is dominant: {}".format(dominant_counts.get('Pixel Error', 0)))
    print("   - Percentage of points where Pixel Error is dominant: {:.2f}%".format(dominant_percentages.get('Pixel Error', 0)))


    # Create a scatter plot of actual error vs cancellation
    # Create a scatter plot using Plotly





    # Create a new figure for the error components analysis
    fig = go.Figure()

    # Add a bar trace for Pixel Error
    fig.add_trace(go.Bar(
        name='Pixel Error',
        x=df_pond.index,  # Use the index of df_pond for x-axis
        y=df_pond['pixel_error_mm'],  # Use pixel_error_mm from df_pond
        marker_color='#1f77b4',  # Set the color for Pixel Error bars
        hovertemplate="Image: %{customdata[0]}<br>" +
                      "Prawn ID: %{customdata[1]}<br>" +
                      "Mean Length: %{customdata[2]:.1f}mm<br>" +
                      "Model Length: %{customdata[3]:.1f}mm<br>" +
                      "Pixel Error: %{y:.1f}mm<br>" +
                      "Impact: %{customdata[4]:.1f}%<extra></extra>",
        customdata=df_pond[['Label', 'PrawnID', 'mean_length', 'Length_fov(mm)']].values  # Use df_pond for custom data
    ))

    # Add a bar trace for Scale Error
    fig.add_trace(go.Bar(
        name='Scale Error',
        x=df_pond.index,  # Use the index of df_pond for x-axis
        y=df_pond['scale_error_mm'],  # Use scale_error_mm from df_pond
        marker_color='#ff7f0e',  # Set the color for Scale Error bars
        hovertemplate="Image: %{customdata[0]}<br>" +
                      "Prawn ID: %{customdata[1]}<br>" +
                      "Mean Length: %{customdata[2]:.1f}mm<br>" +
                      "Model Length: %{customdata[3]:.1f}mm<br>" +
                      "Scale Error: %{y:.1f}mm<br>" +
                      "Impact: %{customdata[4]:.1f}%<extra></extra>",
        customdata=df_pond[['Label', 'PrawnID', 'mean_length', 'Length_fov(mm)']].values  # Use df_pond for custom data
    ))

    # Update the layout of the figure
    fig.update_layout(
        barmode='relative',  # Display bars side by side
        title=f'Error Components Analysis - {pond_type}',  # Dynamic title based on pond type
        xaxis_title='Measurement Index',  # X-axis label
        yaxis_title='Error (mm)',  # Y-axis label
        showlegend=True,  # Display legend
        width=1000,  # Set the width of the figure
        height=600  # Set the height of the figure
    )
    # Show the plot
    fig.show()
    outfile = out_root / f"error_components_{pond_type}.png"
    out_html = out_root / f"error_components_{pond_type}.html"
    fig.write_html(out_html)
    fig.write_image(outfile, width=1000, height=600, scale=2)
    


In [18]:
import pyperclip
pyperclip.copy(cap.stdout)     # copies everything that was printed
print("✓ copied {:,} characters".format(len(cap.stdout)))

✓ copied 2,664 characters


### scale error ###

In [19]:


from pathlib import Path

out_root = Path("/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/phd/"
                "paper/images/final images/body")
out_root.mkdir(parents=True, exist_ok=True)



for pond_type in df['Pond_Type'].unique():
    df_pond = df[df['Pond_Type'] == pond_type]
    df_pond['pixel_error_mm'] = (df_pond['mean_pixels'] - df_pond['pred_Distance_pixels']) * (1/df_pond['mean_scale']) * 10
    df_pond['scale_error_mm'] = df_pond['mean_pixels'] * (1/df_pond['mean_scale'] - 1/df_pond['pred_scale']) * 10

    # Magnitude of scale error
    df_pond['scale_error_mm'] = abs(df_pond['scale_error_mm'])
    
    # Mean and standard deviation of scale error
    mean_scale_error = df_pond['scale_error_mm'].mean()
    std_scale_error = df_pond['scale_error_mm'].std()
    print(f"\n--- Scale Error Analysis for Pond Type: {pond_type} ---")
    print(f"   - Mean of Scale Error: {mean_scale_error:.2f} mm")
    print(f"   - Standard Deviation of Scale Error: {std_scale_error:.2f} mm")
    
    # Mean and standard deviation relative to scale error
    mean_relative_scale_error = (mean_scale_error / df_pond['Length_fov(mm)']).abs().mean()*100
    std_relative_scale_error = (std_scale_error / df_pond['Length_fov(mm)']).abs().mean()*100   
    print(f"   - Mean of Scale Error Relative to Length: {mean_relative_scale_error:.2f}%")
    print(f"   - Standard Deviation of Scale Error Relative to Length: {std_relative_scale_error:.2f}%")
    print("\n----------------------------------------\n")


    #shared scale error in the same image
    df_pond['scale_error_percentage'] = (df_pond['scale_error_mm'] / df_pond['Length_fov(mm)']) * 100
    shared_label_count = df_pond[df_pond['scale_error_percentage'] > 15]['Label'].duplicated(keep=False).sum()
    percent_shared = 100 * shared_label_count / len(df_pond)
    print(f"Percentage of scale error points that share the same image (Label) with at least one other scale error point with at least 15% error (all samples): {percent_shared:.4f}%")
    print(f"Number of scale error points that share the same image (Label) with at least one other scale error point with at least 15% error (all samples): {shared_label_count}")
    print(f"Total number of images with shared scale error with at least 15% error: {df_pond[df_pond['scale_error_percentage'] >15]['Label'].nunique()} out of {len(df_pond['Label'].unique())}")
    print("\n----------------------------------------\n")


    # ---------- GROUP BY IMAGE ----------------------------------------------
    df_img = (df_pond
            .groupby(
                df_pond['Label'].apply(
                    lambda x: tuple(x.split('undistorted_')[-1].split('_')[0:3])
                )
            )
            .agg(scale_err_perc = ('scale_error_percentage', 'mean'),
                prawn_count    = ('scale_error_percentage', 'size'))   # <-- NEW
            .reset_index())

    df_img = df_img.sort_values('scale_err_perc', ascending=False).reset_index(drop=True)

    # ---------- PLOT ---------------------------------------------------------
    fig = go.Figure()

    # Add bar trace
    fig.add_trace(go.Bar(
        x=df_img.index,
        y=df_img['scale_err_perc'],
        text=df_img['prawn_count'],  # Add prawn count as text
        textposition='inside',  # Position text inside the bars
        marker=dict(
            color=df_img['scale_err_perc'],  # Use scale error percentage for color
            colorscale='Viridis',  # Apply a color scale
            showscale=True  # Show color scale
        ),
        hoverinfo='x+y+text',  # Show x, y, and text on hover
    ))

    # Update layout
    fig.update_layout(
        title='Per-image scale error (color = scale error, white number = prawn count)',
        xaxis=dict(
            title='Image (Label)',
            tickvals=df_img.index,
            ticktext=df_img['Label'],
            tickangle=90
        ),
        yaxis=dict(
            title='Scale error (%)'
        ),
        width=1200,
        height=400,
        showlegend=False
    )

    # Show the plot
    # fig.show()

    outfile = out_root / f'{pond_type}_scale_error_per_image.png'
    fig.write_image(outfile, width=1000, height=600, scale=2)
# copies everything that was printed



--- Scale Error Analysis for Pond Type: circle_female ---
   - Mean of Scale Error: 10.35 mm
   - Standard Deviation of Scale Error: 6.97 mm
   - Mean of Scale Error Relative to Length: 8.26%
   - Standard Deviation of Scale Error Relative to Length: 5.57%

----------------------------------------

Percentage of scale error points that share the same image (Label) with at least one other scale error point with at least 15% error (all samples): 10.8696%
Number of scale error points that share the same image (Label) with at least one other scale error point with at least 15% error (all samples): 10
Total number of images with shared scale error with at least 15% error: 6 out of 33

----------------------------------------


--- Scale Error Analysis for Pond Type: circle_male ---
   - Mean of Scale Error: 13.99 mm
   - Standard Deviation of Scale Error: 5.72 mm
   - Mean of Scale Error Relative to Length: 21.75%
   - Standard Deviation of Scale Error Relative to Length: 8.89%

----------

### pixel error ###

In [20]:


out_root = Path("/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/phd/"
                "paper/images/final images/body")
out_root.mkdir(parents=True, exist_ok=True)


for pond_type in df['Pond_Type'].unique():
    df_pond = df[df['Pond_Type'] == pond_type]
    df_pond['pixel_error_mm'] = abs(abs(df_pond['mean_pixels'] - df_pond['pred_Distance_pixels']) * (1/df_pond['mean_scale']) * 10)
    df_pond['scale_error_mm'] = abs(df_pond['mean_pixels'] * (1/df_pond['mean_scale'] - 1/df_pond['pred_scale']) * 10)
    
    df_pond['annotation_pixel_error'] = abs(df_pond['pred_Distance_pixels'] - df_pond['Length_ground_truth_annotation_pixels'])
    df_pond['annotation_pixel_error_mm'] = abs(df_pond['annotation_pixel_error'] * (1/df_pond['mean_scale']) * 10)
    df_pond['annotation_pixel_error_smaller'] = df_pond['annotation_pixel_error_mm'] < abs(df_pond['pixel_error_mm'])
    
    # Print beautifully formatted output
    print(f"\n--- Analysis for Pond Type: {pond_type} ---")
    print(f"Percentage of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): {df_pond['annotation_pixel_error_smaller'].mean()*100:.4f}%")
    print(f"Number of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): {df_pond['annotation_pixel_error_smaller'].sum()}")
    print(f"Mean pixel error: {df_pond['pixel_error_mm'].mean():.2f} mm")
    print(f"Mean annotation pixel error: {df_pond['annotation_pixel_error_mm'].mean():.2f} mm")
    print("----------------------------------------\n")


    df_long = (df_pond[['pixel_error_mm', 'annotation_pixel_error_mm']]
               .rename(columns={'pixel_error_mm': 'Imagej pixel error', 
                                'annotation_pixel_error_mm': 'Roboflow annotation pixel error'})
               .melt(var_name='Error_type', value_name='Error_px'))

    # Create a Plotly box plot
    fig = go.Figure()

    # Add a single box plot trace for combined errors
    fig.add_trace(go.Box(
        y=df_long[df_long['Error_type'] == 'Imagej pixel error']['Error_px'],
        x=df_long[df_long['Error_type'] == 'Imagej pixel error']['Error_type'],
        name='Imagej Pixel Error',
        marker_color='rgb(214, 39, 40)'  # Red color for Imagej pixel error
    ))
    fig.add_trace(go.Box(
        y=df_long[df_long['Error_type'] == 'Roboflow annotation pixel error']['Error_px'],
        x=df_long[df_long['Error_type'] == 'Roboflow annotation pixel error']['Error_type'],
        name='Roboflow Annotation Pixel Error',
        marker_color='rgb(31, 119, 180)'  # Blue color for Roboflow annotation pixel error
    ))

    # Update layout
    fig.update_layout(
        title=f'Imagej pixel-detection error vs Roboflow annotation error for {pond_type}',
        yaxis_title='Error (pixels)',
        xaxis_title='Error Type',
        width=800,
        height=600
    )

    outfile = out_root / f'{pond_type}_combined_error_comparison.png'
    fig.write_image(outfile, width=1000, height=600, scale=2)


--- Analysis for Pond Type: circle_female ---
Percentage of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): 55.4348%
Number of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): 51
Mean pixel error: 6.61 mm
Mean annotation pixel error: 6.30 mm
----------------------------------------


--- Analysis for Pond Type: circle_male ---
Percentage of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): 84.6154%
Number of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): 11
Mean pixel error: 10.01 mm
Mean annotation pixel error: 7.31 mm
----------------------------------------


--- Analysis for Pond Type: square ---
Percentage of points where the Roboflow annotation pixel error is smaller than Imagej pixel error (all pixel error samples): 42.8571%
Number 

In [21]:
for pond_type in df['Pond_Type'].unique():
        print(f"Mean length for {pond_type}: {df[df['Pond_Type'] == pond_type]['mean_length'].mean()}")
        print(f"Std length for {pond_type}: {df[df['Pond_Type'] == pond_type]['std_length'].mean()}")
        print(f"Number of prawns in {pond_type}: {len(df[df['Pond_Type'] == pond_type])}")
        df_pond = df[df['Pond_Type'] == pond_type]

        min_val = min(df_pond['mean_length'].min(), df_pond['Length_fov(mm)'].min())
        max_val = max(df_pond['mean_length'].max(), df_pond['Length_fov(mm)'].max())
        x = np.linspace(min_val, max_val, 100)

        median_mad = df_pond['std_length'].mean()

        # Create scatter plot
        scatter = go.Scatter(
            x=df_pond['mean_length'],
            y=df_pond['Length_fov(mm)'],
            mode='markers',
            name='Measurements',
            text=df_pond.apply(lambda row: f"Image: {row['Label']}<br>Prawn ID: {row['PrawnID']}", axis=1),
            hoverinfo='text+x+y',
            marker=dict(
                color='#ff7f0e',
                opacity=0.7
            )
        )

       

        # Create diagonal lines
        diag = go.Scatter(x=x, y=x, mode='lines', name='Perfect match (y=x)', 
                        line=dict(color='black', dash='dash'))
        upper = go.Scatter(x=x, y=x + median_mad, mode='lines', 
                        name=f'mean + Std (variability) ({median_mad:.2f} mm)',
                        line=dict(color='#2c7fb8', dash='dash'))
        lower = go.Scatter(x=x, y=x - median_mad, mode='lines',
                        name=f'mean - Std (variability) ({median_mad:.2f} mm)', 
                        line=dict(color='#2c7fb8', dash='dash'))

        # Calculate percentage within bounds for this pond type
        within_bounds = ((df_pond['Length_fov(mm)'] <= df_pond['mean_length'] + median_mad) & 
                        (df_pond['Length_fov(mm)'] >= df_pond['mean_length'] - median_mad)).mean() * 100

        # Create layout
        layout = go.Layout(
            title=f'Model Measurements vs mean Values with Std (variability) Boundaries - {pond_type}',
            xaxis_title='mean of manual measurements (mm)',
            yaxis_title='Model Measurements  (mm)',
            showlegend=True,
            width=800,
            height=800
        )

        # Create figure and add traces
        fig = go.Figure(data=[scatter, diag, upper, lower], layout=layout)

        # Update layout for equal aspect ratio
        fig.update_layout(
            yaxis=dict(
                scaleanchor="x",
                scaleratio=1
            )
        )
        fig.show()

Mean length for circle_female: 116.7372862318841
Std length for circle_female: 7.582457260895332
Number of prawns in circle_female: 92


Mean length for circle_male: 77.39110256410257
Std length for circle_male: 5.624595287016238
Number of prawns in circle_male: 13


Mean length for square: 119.58761904761907
Std length for square: 7.729485531879914
Number of prawns in square: 14


In [25]:
for pond_type in df['Pond_Type'].unique():
    #histogram of meAN absolute error
    df_pond = df[df['Pond_Type'] == pond_type]
    df_pond['mean_absolute_error'] =abs(df_pond['mean_length'] - df_pond['Length_fov(mm)'])
    #histogram of mean absolute error    

    #histogram of mean absolute error
    fig = px.histogram(df_pond, x='mean_absolute_error', nbins=100, title=f'Histogram of mean absolute error for {pond_type}')
    fig.show()